In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.font_manager as fm
import matplotlib.gridspec as gridspec
import math
from influxdb_client import InfluxDBClient


In [2]:
from common import (
    sensor_types, sensor_type_mapping,
    get_influx_client
)


# InfluxDB 연결
client, INFLUX_BUCKET = get_influx_client()
read_api = client.query_api()

# 센서 타입 필터 생성
sensor_filter = ' or '.join([f'r["type"] == "{t}"' for t in sensor_types])

# Flux 쿼리 작성
flux = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: -1d)
  |> filter(fn: (r) => r["_measurement"] == "sensor")
  |> filter(fn: (r) => r["_field"] == "value")
  |> filter(fn: (r) => {sensor_filter})
  |> aggregateWindow(every: 5m, fn: mean, createEmpty: false)
  |> yield(name: "mean")
'''

# 데이터 쿼리 및 전처리
df = read_api.query_data_frame(flux)

if isinstance(df, list):
    df = pd.concat(df)

df = df.reset_index().rename(columns={"_time": "time"})
df = df.set_index("time")

# 데이터 확인
print("데이터 샘플:")
display(df.head())
print(f"데이터 크기: {df.shape}")


/opt/homebrew/anaconda3/lib/python3.12/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
from(bucket: "coffee-mqtt")
  |> range(start: -1d)
  |> filter(fn: (r) => r["_measurement"] == "sensor")
  |> filter(fn: (r) => r["_field"] == "value")
  |> filter(fn: (r) => r["type"] == "co2" or r["type"] == "distance" or r["type"] == "humidity" or r["type"] == "illumination" or r["type"] == "infrared" or r["type"] == "pressure" or r["type"] == "temperature" or r["type"] == "infrared_and_visible" or r["type"] == "occupancy" or r["type"] == "battery_level" or r["type"] == "activity" or r["type"] == "battery")
  |> aggregateWindow(every: 5m, fn: mean, createEmpty: false)
  |> yield(name: "mean")
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnin

데이터 샘플:


,index,result,table,_start,_stop,_value,_field,_measurement,location,type
time,,,,,,,,,,
2025-04-24 01:50:00+00:00,0,mean,0,2025-04-23 08:09:19.328503+00:00,2025-04-24 08:09:19.328503+00:00,32.0,value,sensor,2,battery
2025-04-24 01:55:00+00:00,1,mean,0,2025-04-23 08:09:19.328503+00:00,2025-04-24 08:09:19.328503+00:00,32.0,value,sensor,2,battery
2025-04-24 02:00:00+00:00,2,mean,0,2025-04-23 08:09:19.328503+00:00,2025-04-24 08:09:19.328503+00:00,32.0,value,sensor,2,battery
2025-04-24 02:05:00+00:00,3,mean,0,2025-04-23 08:09:19.328503+00:00,2025-04-24 08:09:19.328503+00:00,32.0,value,sensor,2,battery
2025-04-24 02:10:00+00:00,4,mean,0,2025-04-23 08:09:19.328503+00:00,2025-04-24 08:09:19.328503+00:00,32.0,value,sensor,2,battery


데이터 크기: (1759, 10)


In [3]:
df = df[df["location"]!="냉장고 내부"]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1730 entries, 2025-04-24 01:50:00+00:00 to 2025-04-24 08:09:19.328503+00:00
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   index         1730 non-null   int64              
 1   result        1730 non-null   object             
 2   table         1730 non-null   int64              
 3   _start        1730 non-null   datetime64[ns, UTC]
 4   _stop         1730 non-null   datetime64[ns, UTC]
 5   _value        1730 non-null   float64            
 6   _field        1730 non-null   object             
 7   _measurement  1730 non-null   object             
 8   location      1730 non-null   object             
 9   type          1730 non-null   object             
dtypes: datetime64[ns, UTC](2), float64(1), int64(2), object(5)
memory usage: 148.7+ KB


# 장소별 데이터 분석